In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import cv2
import numpy as np
import random
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report

In [ ]:
!git clone https://github.com/haianh23102000/hoc_may.git

fatal: destination path 'hoc_may' already exists and is not an empty directory.


In [ ]:
def get_clases():
  classes = ['2C', '3C', '4C']
  return classes
TrainTest = namedtuple('TrainTest', ['train', 'test'])

In [ ]:
def prepare_data():
  gaussianBlur = transforms.GaussianBlur(3)
  histogramEqualize = transforms.RandomEqualize(p=0.5)
  horizontal = transforms.RandomHorizontalFlip()
  vertical = transforms.RandomVerticalFlip()
  resize_224 = transforms.Resize((224,224))
  crop_224 = transforms.RandomCrop(224, padding=4)
  resize_32 = transforms.Resize((32,32))
  crop_32 = transforms.RandomCrop(32, padding=4)
  tensor = transforms.ToTensor()
  transform_train_raw224 = transforms.Compose([resize_224, tensor])
  transform_train_aug224 = transforms.Compose([resize_224, crop_224, horizontal, vertical, tensor])
  transform_train_pre224 = transforms.Compose([resize_224, gaussianBlur, histogramEqualize, tensor])
  transform_test_224 = transforms.Compose([resize_224, tensor])
  transform_train_raw232 = transforms.Compose([resize_32, tensor])
  transform_train_aug32 = transforms.Compose([resize_32, crop_32, horizontal, vertical, tensor])
  transform_train_pre32 = transforms.Compose([resize_32, gaussianBlur, histogramEqualize, tensor])
  transform_test_32 = transforms.Compose([resize_32, tensor])
  trainset = torchvision.datasets.ImageFolder(root=traindir, transform=transform_train_aug224)
  testset = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test_224)
  return TrainTest(train=trainset, test=testset)

In [ ]:
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=35, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=35, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)

In [ ]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  true = []
  pred = []
  model.train()
  running_loss = 0.0
  reporting_steps = 18
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    true += list(labels.cpu().numpy())
    _, predicted = torch.max(outputs, dim=1)
    pred += list(predicted.cpu().numpy())
    loss = loss_func(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  return pred, true

In [ ]:
def test_epoch(epoch, model, loader, device):
  true = []
  pred = [] 
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      true += list(labels.cpu().numpy())
      pred += list(predicted.cpu().numpy())
  return pred, true

In [ ]:
def main(PATH='./model.pth', model_in=''):
  classes = get_clases()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0")
  torch.cuda.empty_cache()
  if model_in == 'vgg16':
    print("vgg16")
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=3, bias=True)
    model.to(device=device)
  elif model_in == 'resnet50':
    print("resnet50")
    model = torchvision.models.resnet50(pretrained=False, progress=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=3, bias=True)  
    model.to(device=device)
  elif model_in == 'desnet':
    print("desnet")
    model = torchvision.models.densenet121(pretrained=False, progress=False)
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=3, bias=True)
    model.to(device=device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    pred, true = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(true, pred, target_names=classes))
    torch.save(model.state_dict(), PATH)
  return model
model = main(PATH="./vgg16.pth", model_in='vgg16')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


vgg16
Epoch 0 step 17 ave_loss 1.0968
Epoch 0 step 35 ave_loss 1.0992
Epoch 0 step 53 ave_loss 1.0896
Epoch 0 step 71 ave_loss 1.0997
Epoch 0 step 89 ave_loss 1.0956
Epoch 0 step 107 ave_loss 1.0925
Epoch 0 step 125 ave_loss 1.0850
Epoch 0 step 143 ave_loss 1.0937
Epoch 0 step 161 ave_loss 1.0905
Epoch 0 step 179 ave_loss 1.0787


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          2C       0.38      0.58      0.46       409
          3C       0.00      0.00      0.00       367
          4C       0.59      0.70      0.64       831

    accuracy                           0.51      1607
   macro avg       0.32      0.43      0.37      1607
weighted avg       0.40      0.51      0.45      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

Epoch 2 step 17 ave_loss 1.0165
Epoch 2 step 35 ave_loss 0.8571
Epoch 2 step 53 ave_loss 0.9007
Epoch 2 step 71 ave_loss 0.8082
Epoch 2 step 89 ave_loss 0.9389
Epoch 2 step 107 ave_loss 0.8082
Epoch 2 step 125 ave_loss 0.7610
Epoch 2 step 143 ave_loss 0.6933
Epoch 2 step 161 ave_loss 0.7015
Epoch 2 step 179 ave_loss 0.4587
              precision    recall  f1-score   support

          2C       0.44      0.58      0.50       409
          3C       0.41      0.63      0.50       367
          4C       0.89      0.54      0.67       831

    accuracy                           0.57      1607
   macro avg       0.58      0.58      0.56      1607
weighted avg       0.67      0.57      0.59      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 3 step 17 ave_loss 0.7955
Epoch 3 step 35 ave_loss 0.5315
Epoch 3 step 53 ave_loss 0.5374
Epoch 3 step 71 ave_loss 0.5043
Epoch 3 step 89 ave_loss 0.5198
Epoch 3 step 107 ave_loss 0.4477
Epoch 3 step 125 ave_loss 0.5052
Epoch 3 step 143 ave_loss 0.4148
Epoch 3 step 161 ave_loss 0.3738
Epoch 3 step 179 ave_loss 0.3288
              precision    recall  f1-score   support

          2C       0.90      0.68      0.77       409
          3C       0.54      0.82      0.65       367
          4C       0.94      0.84      0.89       831

    accuracy                           0.80      1607
   macro avg       0.79      0.78      0.77      1607
weighted avg       0.84      0.80      0.81      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 4 step 17 ave_loss 0.4406
Epoch 4 step 35 ave_loss 0.3102
Epoch 4 step 53 ave_loss 0.2679
Epoch 4 step 71 ave_loss 0.2389
Epoch 4 step 89 ave_loss 0.2709
Epoch 4 step 107 ave_loss 0.1393
Epoch 4 step 125 ave_loss 0.2126
Epoch 4 step 143 ave_loss 0.2171


In [ ]:
traindir = "/content/hoc_may/Data/train"
testdir = "/content/hoc_may/Data/test"

In [ ]:
def vid(model=None, testdir=None, device='cuda'):
  if testdir == None:
    testdir = torchvision.datasets.ImageFolder(root=testdir,transform=ransforms.Compose([resize_224, transforms.ToTensor()]))
  video = namedtuple('video', ['id', "label_true", 'label_pred'])
  id_list = []
  for i, image in enumerate(testdir.imgs):
    id = image[0].split("/")[-1].split("_")[0]
    id_list.append(id)
  vid_list = np.unique(id_list, return_counts=False)
  true = []
  pred = []
  model.to(device)
  with torch.no_grad():
    model.eval()
    for images, labels in testdir:
      images = images.unsqueeze(0).to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      true.append(labels)
      pred += list(predicted.cpu().numpy())
  outputs = []
  true_vid = []
  pred_vid = []
  for id in vid_list:
    true_vid = []
    pred_vid = []
    for index, img in enumerate(id_list):
      if img == id:
        pred_vid.append(pred[index])
        true_vid.append(true[index])
    value_true, count_true = np.unique(true_vid, return_counts=True)
    label_true = value_true[np.where(count_true == np.max(count_true))]
    value_pred, count_pred = np.unique(pred_vid, return_counts=True)
    label_pred = value_pred[np.where(count_pred == np.max(count_pred))]
    print("id:", id, ", true:", label_true, ", pred:",label_pred)
    true_video.append(label_true)
    pred_video.append(label_pred)
    outputs.append(video(id=id, label_true=label_true, label_pred=label_pred))
  classes = get_clases()
  print(classification_report(true_vid, pred_vid, target_names=classes))
  return outputs

In [ ]:
vid(model=model, testdir=prepare_data().test, device='cuda')

id: 157 , true: [1] , pred: [1]
id: 158 , true: [0] , pred: [0]
id: 159 , true: [1] , pred: [1]
id: 160 , true: [2] , pred: [2]
id: 161 , true: [1] , pred: [1]
id: 162 , true: [1] , pred: [1]
id: 163 , true: [2] , pred: [2]
id: 164 , true: [2] , pred: [2]
id: 165 , true: [0] , pred: [0]
id: 166 , true: [1] , pred: [1]
id: 167 , true: [2] , pred: [2]
id: 168 , true: [0] , pred: [0]
id: 169 , true: [0] , pred: [0]
id: 170 , true: [2] , pred: [1]
id: 171 , true: [0] , pred: [0]
id: 172 , true: [2] , pred: [2]
id: 173 , true: [2] , pred: [0]
id: 174 , true: [1] , pred: [0]
id: 175 , true: [1] , pred: [1]
id: 176 , true: [0] , pred: [0]
id: 177 , true: [0] , pred: [0]
id: 178 , true: [0] , pred: [0]
id: 179 , true: [1] , pred: [0]
id: 180 , true: [2] , pred: [2]
id: 181 , true: [0] , pred: [1]
id: 182 , true: [2] , pred: [2]
id: 183 , true: [0] , pred: [0]
id: 184 , true: [2] , pred: [2]
id: 185 , true: [1] , pred: [0]
id: 186 , true: [1] , pred: [1]
id: 187 , true: [2] , pred: [2]
id: 188 

[video(id='157', label_true=array([1]), label_pred=array([1])),
 video(id='158', label_true=array([0]), label_pred=array([0])),
 video(id='159', label_true=array([1]), label_pred=array([1])),
 video(id='160', label_true=array([2]), label_pred=array([2])),
 video(id='161', label_true=array([1]), label_pred=array([1])),
 video(id='162', label_true=array([1]), label_pred=array([1])),
 video(id='163', label_true=array([2]), label_pred=array([2])),
 video(id='164', label_true=array([2]), label_pred=array([2])),
 video(id='165', label_true=array([0]), label_pred=array([0])),
 video(id='166', label_true=array([1]), label_pred=array([1])),
 video(id='167', label_true=array([2]), label_pred=array([2])),
 video(id='168', label_true=array([0]), label_pred=array([0])),
 video(id='169', label_true=array([0]), label_pred=array([0])),
 video(id='170', label_true=array([2]), label_pred=array([1])),
 video(id='171', label_true=array([0]), label_pred=array([0])),
 video(id='172', label_true=array([2]), 